# Aditya-L1 CME Detection - Threshold Optimization

This notebook focuses on optimizing detection thresholds for CME events using statistical analysis and machine learning techniques.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, roc_curve, auc, classification_report
from sklearn.model_selection import GridSearchCV
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
print("Threshold Optimization for Aditya-L1 CME Detection")
print("=" * 55)

## 1. Load Data and Prepare Features

In [ ]:
# Load data (using simulation for demonstration)
import sys
sys.path.append('../')

from src.data_ingestion.swis_ingestion import SWISIngestion
from src.data_ingestion.cactus_ingestion import CACTUSIngestion
from src.processing.feature_engineering import FeatureEngineer
from src.config import config

# Generate sample data
swis_ingestion = SWISIngestion()
cactus_ingestion = CACTUSIngestion()

# Simulate 30 days of data for better statistics
particle_data = swis_ingestion.simulate_particle_data(duration_minutes=30*24*60)
particle_df = pd.DataFrame(particle_data)
particle_df['timestamp'] = pd.to_datetime(particle_df['timestamp'])

cme_events = cactus_ingestion.simulate_cme_events(days=30)
cme_df = pd.DataFrame(cme_events)
cme_df['timestamp'] = pd.to_datetime(cme_df['timestamp'])

print(f"Loaded {len(particle_df)} particle data points and {len(cme_df)} CME events")

In [ ]:
# Engineer features
feature_engineer = FeatureEngineer(config._config)
features_df = feature_engineer.engineer_features(particle_df)

print(f"Engineered features shape: {features_df.shape}")
print(f"Feature columns: {len(features_df.columns)}")

## 2. Create Labels for Training

In [ ]:
# Create binary labels for CME detection
from datetime import timedelta

def create_cme_labels(features_df, cme_df, window_minutes=30):
    """
    Create binary labels for CME detection
    """
    labels = np.zeros(len(features_df))
    
    for _, cme_event in cme_df.iterrows():
        cme_time = cme_event['timestamp']
        
        # Define time window around CME event
        start_time = cme_time - timedelta(minutes=window_minutes)
        end_time = cme_time + timedelta(minutes=window_minutes)
        
        # Mark data points within this window as CME events
        mask = (features_df['timestamp'] >= start_time) & (features_df['timestamp'] <= end_time)
        labels[mask] = 1
    
    return labels

# Create labels
labels = create_cme_labels(features_df, cme_df)

print(f"Created labels: {len(labels)} total, {np.sum(labels)} CME events ({np.mean(labels)*100:.2f}%)")
print(f"Class distribution: {np.bincount(labels.astype(int))}")

## 3. Single Threshold Optimization

In [ ]:
# Optimize single parameter thresholds
def optimize_single_threshold(feature_values, labels, metric='f1'):
    """
    Optimize threshold for a single feature
    """
    thresholds = np.percentile(feature_values, np.linspace(50, 99, 50))
    scores = []
    
    for threshold in thresholds:
        predictions = (feature_values >= threshold).astype(int)
        
        # Calculate metrics
        tp = np.sum((predictions == 1) & (labels == 1))
        fp = np.sum((predictions == 1) & (labels == 0))
        fn = np.sum((predictions == 0) & (labels == 1))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        if metric == 'f1':
            scores.append(f1)
        elif metric == 'precision':
            scores.append(precision)
        elif metric == 'recall':
            scores.append(recall)
    
    best_idx = np.argmax(scores)
    return thresholds[best_idx], scores[best_idx], thresholds, scores

# Test key features
key_features = ['proton_flux', 'velocity', 'temperature', 'magnetic_field_magnitude']
threshold_results = {}

for feature in key_features:
    if feature in features_df.columns:
        best_threshold, best_score, thresholds, scores = optimize_single_threshold(
            features_df[feature].values, labels
        )
        threshold_results[feature] = {
            'best_threshold': best_threshold,
            'best_score': best_score,
            'thresholds': thresholds,
            'scores': scores
        }
        print(f"{feature}: Best threshold = {best_threshold:.2f}, F1 score = {best_score:.3f}")

In [ ]:
# Plot threshold optimization curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Single Feature Threshold Optimization', fontsize=16)

for i, (feature, results) in enumerate(threshold_results.items()):
    row, col = i // 2, i % 2
    
    axes[row, col].plot(results['thresholds'], results['scores'], 'b-', linewidth=2)
    axes[row, col].axvline(results['best_threshold'], color='red', linestyle='--', 
                          label=f'Best: {results["best_threshold"]:.2f}')
    axes[row, col].set_title(f'{feature.replace("_", " ").title()}')
    axes[row, col].set_xlabel('Threshold')
    axes[row, col].set_ylabel('F1 Score')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Multi-Parameter Threshold Optimization

In [ ]:
# Multi-parameter threshold optimization
def multi_threshold_objective(thresholds, features_matrix, labels, weights=None):
    """
    Objective function for multi-parameter threshold optimization
    """
    if weights is None:
        weights = np.ones(len(thresholds))
    
    # Apply thresholds
    predictions = np.zeros(len(labels))
    
    for i, threshold in enumerate(thresholds):
        feature_predictions = (features_matrix[:, i] >= threshold).astype(float)
        predictions += weights[i] * feature_predictions
    
    # Convert to binary predictions
    predictions = (predictions >= np.sum(weights) * 0.5).astype(int)
    
    # Calculate F1 score
    tp = np.sum((predictions == 1) & (labels == 1))
    fp = np.sum((predictions == 1) & (labels == 0))
    fn = np.sum((predictions == 0) & (labels == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return -f1  # Negative because we're minimizing

# Prepare feature matrix
selected_features = ['proton_flux', 'velocity', 'temperature']
available_features = [f for f in selected_features if f in features_df.columns]

if available_features:
    features_matrix = features_df[available_features].values
    
    # Initial thresholds (median values)
    initial_thresholds = [np.median(features_matrix[:, i]) for i in range(len(available_features))]
    
    # Bounds for optimization (10th to 90th percentile)
    bounds = [(np.percentile(features_matrix[:, i], 10), 
              np.percentile(features_matrix[:, i], 90)) 
             for i in range(len(available_features))]
    
    # Optimize
    result = minimize(
        multi_threshold_objective,
        initial_thresholds,
        args=(features_matrix, labels),
        bounds=bounds,
        method='L-BFGS-B'
    )
    
    optimal_thresholds = result.x
    optimal_f1 = -result.fun
    
    print("Multi-parameter Optimization Results:")
    print(f"Optimal F1 Score: {optimal_f1:.3f}")
    for i, feature in enumerate(available_features):
        print(f"{feature}: {optimal_thresholds[i]:.2f}")
else:
    print("No suitable features available for multi-parameter optimization")

## 5. ROC and Precision-Recall Analysis

In [ ]:
# ROC and PR curve analysis
def plot_roc_pr_curves(features_df, labels, feature_name):
    """
    Plot ROC and Precision-Recall curves for a feature
    """
    if feature_name not in features_df.columns:
        print(f"Feature {feature_name} not found")
        return
    
    feature_values = features_df[feature_name].values
    
    # ROC curve
    fpr, tpr, roc_thresholds = roc_curve(labels, feature_values)
    roc_auc = auc(fpr, tpr)
    
    # Precision-Recall curve
    precision, recall, pr_thresholds = precision_recall_curve(labels, feature_values)
    pr_auc = auc(recall, precision)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # ROC curve
    ax1.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
    ax1.plot([0, 1], [0, 1], 'r--', alpha=0.5)
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title(f'ROC Curve - {feature_name.replace("_", " ").title()}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Precision-Recall curve
    ax2.plot(recall, precision, 'g-', linewidth=2, label=f'PR (AUC = {pr_auc:.3f})')
    ax2.axhline(y=np.mean(labels), color='r', linestyle='--', alpha=0.5, label='Baseline')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title(f'Precision-Recall Curve - {feature_name.replace("_", " ").title()}')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return roc_auc, pr_auc

# Analyze key features
auc_results = {}
for feature in ['proton_flux', 'velocity']:
    if feature in features_df.columns:
        print(f"\nAnalyzing {feature}:")
        roc_auc, pr_auc = plot_roc_pr_curves(features_df, labels, feature)
        auc_results[feature] = {'roc_auc': roc_auc, 'pr_auc': pr_auc}
        print(f"ROC AUC: {roc_auc:.3f}, PR AUC: {pr_auc:.3f}")

## 6. Threshold Sensitivity Analysis

In [ ]:
# Sensitivity analysis for threshold variations
def threshold_sensitivity_analysis(features_df, labels, base_thresholds, feature_names, variation_range=0.2):
    """
    Analyze sensitivity of performance to threshold variations
    """
    results = []
    
    for i, feature in enumerate(feature_names):
        if feature not in features_df.columns:
            continue
            
        base_threshold = base_thresholds[i]
        variations = np.linspace(
            base_threshold * (1 - variation_range),
            base_threshold * (1 + variation_range),
            21
        )
        
        for variation in variations:
            # Apply threshold
            predictions = (features_df[feature].values >= variation).astype(int)
            
            # Calculate metrics
            tp = np.sum((predictions == 1) & (labels == 1))
            fp = np.sum((predictions == 1) & (labels == 0))
            fn = np.sum((predictions == 0) & (labels == 1))
            tn = np.sum((predictions == 0) & (labels == 0))
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            
            results.append({
                'feature': feature,
                'threshold': variation,
                'threshold_ratio': variation / base_threshold,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'specificity': specificity
            })
    
    return pd.DataFrame(results)

# Perform sensitivity analysis
if 'optimal_thresholds' in locals() and available_features:
    sensitivity_df = threshold_sensitivity_analysis(
        features_df, labels, optimal_thresholds, available_features
    )
    
    # Plot sensitivity results
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Threshold Sensitivity Analysis', fontsize=16)
    
    metrics = ['precision', 'recall', 'f1', 'specificity']
    
    for i, metric in enumerate(metrics):
        row, col = i // 2, i % 2
        
        for feature in available_features:
            feature_data = sensitivity_df[sensitivity_df['feature'] == feature]
            axes[row, col].plot(feature_data['threshold_ratio'], feature_data[metric], 
                              label=feature, linewidth=2)
        
        axes[row, col].axvline(1.0, color='red', linestyle='--', alpha=0.5, label='Base Threshold')
        axes[row, col].set_title(f'{metric.title()}')
        axes[row, col].set_xlabel('Threshold Ratio')
        axes[row, col].set_ylabel(metric.title())
        axes[row, col].legend()
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Skipping sensitivity analysis - no optimal thresholds available")

## 7. False Positive Analysis

In [ ]:
# Analyze false positives
def analyze_false_positives(features_df, labels, thresholds, feature_names):
    """
    Analyze characteristics of false positive detections
    """
    if not feature_names or len(feature_names) == 0:
        print("No features available for false positive analysis")
        return
    
    # Apply thresholds to get predictions
    predictions = np.zeros(len(labels))
    
    for i, feature in enumerate(feature_names):
        if feature in features_df.columns and i < len(thresholds):
            feature_predictions = (features_df[feature].values >= thresholds[i]).astype(float)
            predictions += feature_predictions
    
    # Convert to binary
    predictions = (predictions >= len(feature_names) * 0.5).astype(int)
    
    # Identify false positives
    fp_mask = (predictions == 1) & (labels == 0)
    tp_mask = (predictions == 1) & (labels == 1)
    
    print(f"False Positive Analysis:")
    print(f"Total predictions: {np.sum(predictions)}")
    print(f"True positives: {np.sum(tp_mask)}")
    print(f"False positives: {np.sum(fp_mask)}")
    print(f"False positive rate: {np.sum(fp_mask) / np.sum(predictions) * 100:.1f}%")
    
    if np.sum(fp_mask) > 0 and np.sum(tp_mask) > 0:
        # Compare characteristics
        fp_data = features_df[fp_mask]
        tp_data = features_df[tp_mask]
        
        print("\nCharacteristics comparison (FP vs TP):")
        for feature in feature_names:
            if feature in features_df.columns:
                fp_mean = fp_data[feature].mean()
                tp_mean = tp_data[feature].mean()
                print(f"{feature}: FP={fp_mean:.2f}, TP={tp_mean:.2f}, Ratio={fp_mean/tp_mean:.2f}")
        
        # Plot comparison
        fig, axes = plt.subplots(1, len(feature_names), figsize=(5*len(feature_names), 4))
        if len(feature_names) == 1:
            axes = [axes]
        
        for i, feature in enumerate(feature_names):
            if feature in features_df.columns:
                axes[i].hist(fp_data[feature], bins=20, alpha=0.7, label='False Positives', color='red')
                axes[i].hist(tp_data[feature], bins=20, alpha=0.7, label='True Positives', color='green')
                axes[i].set_title(f'{feature.replace("_", " ").title()}')
                axes[i].set_xlabel('Value')
                axes[i].set_ylabel('Frequency')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Perform false positive analysis
if 'optimal_thresholds' in locals() and available_features:
    analyze_false_positives(features_df, labels, optimal_thresholds, available_features)
else:
    # Use single feature thresholds
    single_thresholds = [threshold_results[f]['best_threshold'] for f in key_features if f in threshold_results]
    single_features = [f for f in key_features if f in threshold_results]
    if single_thresholds:
        analyze_false_positives(features_df, labels, single_thresholds, single_features)

## 8. Recommended Thresholds

In [ ]:
# Generate final threshold recommendations
print("THRESHOLD OPTIMIZATION SUMMARY")
print("=" * 50)

print("\n1. SINGLE FEATURE THRESHOLDS:")
for feature, results in threshold_results.items():
    print(f"   {feature.replace('_', ' ').title()}:")
    print(f"     - Optimal threshold: {results['best_threshold']:.2f}")
    print(f"     - F1 score: {results['best_score']:.3f}")

if 'optimal_thresholds' in locals() and available_features:
    print("\n2. MULTI-PARAMETER THRESHOLDS:")
    print(f"   Combined F1 score: {optimal_f1:.3f}")
    for i, feature in enumerate(available_features):
        print(f"   {feature.replace('_', ' ').title()}: {optimal_thresholds[i]:.2f}")

print("\n3. FEATURE IMPORTANCE (by AUC):")
if auc_results:
    sorted_features = sorted(auc_results.items(), key=lambda x: x[1]['roc_auc'], reverse=True)
    for feature, aucs in sorted_features:
        print(f"   {feature.replace('_', ' ').title()}: ROC AUC = {aucs['roc_auc']:.3f}")

print("\n4. RECOMMENDED CONFIGURATION:")
print("   For production deployment, consider:")
print("   - Use multi-parameter thresholds for better accuracy")
print("   - Implement adaptive thresholds based on solar activity")
print("   - Regular retraining with new data")
print("   - False positive monitoring and feedback loop")

# Save optimized thresholds to config format
config_output = {
    'detection': {
        'thresholds': {}
    }
}

# Add single feature thresholds
for feature, results in threshold_results.items():
    config_key = f"{feature}_threshold"
    config_output['detection']['thresholds'][config_key] = float(results['best_threshold'])

print("\n5. CONFIGURATION OUTPUT:")
import yaml
print(yaml.dump(config_output, default_flow_style=False))